In [2]:
from bpe import get_encoder

text = "Hello!! I'm Andrej Karpathy. It's 2022. w00t :D 🤗"
e = get_encoder()
r = e.encode_and_show_work(text)

print("Original text is:")
print(text)
print("First the text gets pre-tokenized, broken up into chunks, the outcome is:")
print(r['tokens'])
# ['Hello', '!!', ' I', "'m", ' Andrej', ' Karpathy', '.', ' It', "'s", ' 2022', '.', ' w', '00', 't', ' :', 'D', ' 🤗']
print("Then we iterate over each chunk and process them in turn...")
for part in r['parts']:
    print(part)

Original text is:
Hello!! I'm Andrej Karpathy. It's 2022. w00t :D 🤗
First the text gets pre-tokenized, broken up into chunks, the outcome is:
['Hello', '!!', ' I', "'m", ' Andrej', ' Karpathy', '.', ' It', "'s", ' 2022', '.', ' w', '00', 't', ' :', 'D', ' 🤗']
Then we iterate over each chunk and process them in turn...
{'token': 'Hello', 'token_bytes': b'Hello', 'token_translated': 'Hello', 'token_merged': ['Hello'], 'token_ix': [15496]}
{'token': '!!', 'token_bytes': b'!!', 'token_translated': '!!', 'token_merged': ['!!'], 'token_ix': [3228]}
{'token': ' I', 'token_bytes': b' I', 'token_translated': 'ĠI', 'token_merged': ['ĠI'], 'token_ix': [314]}
{'token': "'m", 'token_bytes': b"'m", 'token_translated': "'m", 'token_merged': ["'m"], 'token_ix': [1101]}
{'token': ' Andrej', 'token_bytes': b' Andrej', 'token_translated': 'ĠAndrej', 'token_merged': ['ĠAndre', 'j'], 'token_ix': [10948, 73]}
{'token': ' Karpathy', 'token_bytes': b' Karpathy', 'token_translated': 'ĠKarpathy', 'token_merged'

在 Byte Pair Encoding (BPE) 中，`Ġ` 表示一个空格字符。在 BPE 的处理过程中，空格字符被标记为 `Ġ`，以便明确区分单词之间的空格。因此，`Ġ:` 中的 `Ġ` 表示在冒号之前有一个空格。

例如，在 BPE 分解中：
- `'token': 't', 'token_translated': 't'` 表示单词 `t` 没有空格前缀。
- `'token': ' :', 'token_translated': 'Ġ:'` 表示在冒号前面有一个空格。

所以这个 `Ġ` 只是一个空格字符的标记，用于在分词过程中保持空格的正确位置。

以下是这些字段的含义：

- `token`: 这是实际的标记（token）。在你的例子中，标记是 `t`。
- `token_bytes`: 这是标记的字节表示形式。在你的例子中，`t` 被表示为 `b't'`。
- `token_translated`: 这是标记的翻译版。这个字段有时会显示一些预处理的结果，比如在 BPE 中加入空格字符等。在你的例子中，`t` 没有变化，所以还是 `t`。
- `token_merged`: 这是标记在分词过程中可能合并的部分。BPE 有时会将多个标记合并成一个更大的标记。在你的例子中，`t` 没有合并，所以还是 `['t']`。
- `token_ix`: 这是标记的索引位置。在你的例子中，索引是 `[83]`。


分为两个主要函数，一个专门统计vocabulary，另一个负责合并字符串。

统计词频很暴力，就是遍历vocabulary里每一个元素，利用像两个元素的滑动窗口，挨个组合并且记录频次。找到频次最高的之后进行合并并且输出新的vocabulary。

In [3]:
import re, collections

text = "The aims for this subject is for students to develop an understanding of the main algorithms used in naturallanguage processing, for use in a diverse range of applications including text classification, machine translation, and question answering. Topics to be covered include part-of-speech tagging, n-gram language modelling, syntactic parsing and deep learning. The programming language used is Python, see for more information on its use in the workshops, assignments and installation at home."
# text = 'low '*5 +'lower '*2+'newest '*6 +'widest '*3

'''
先统计词频
'''
def get_vocab(text):
    
    # 初始化为 0
    vocab = collections.defaultdict(int)
    # 去头去尾再根据空格split
    for word in text.strip().split():
        #note: we use the special token </w> (instead of underscore in the lecture) to denote the end of a word
        # 给list中每个元素增加空格，并在最后增加结束符号，同时统计单词出现次数
        vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab
my_vocab = get_vocab(text)
print(my_vocab)


defaultdict(<class 'int'>, {'T h e </w>': 2, 'a i m s </w>': 1, 'f o r </w>': 4, 't h i s </w>': 1, 's u b j e c t </w>': 1, 'i s </w>': 2, 's t u d e n t s </w>': 1, 't o </w>': 2, 'd e v e l o p </w>': 1, 'a n </w>': 1, 'u n d e r s t a n d i n g </w>': 1, 'o f </w>': 2, 't h e </w>': 2, 'm a i n </w>': 1, 'a l g o r i t h m s </w>': 1, 'u s e d </w>': 2, 'i n </w>': 3, 'n a t u r a l l a n g u a g e </w>': 1, 'p r o c e s s i n g , </w>': 1, 'u s e </w>': 2, 'a </w>': 1, 'd i v e r s e </w>': 1, 'r a n g e </w>': 1, 'a p p l i c a t i o n s </w>': 1, 'i n c l u d i n g </w>': 1, 't e x t </w>': 1, 'c l a s s i f i c a t i o n , </w>': 1, 'm a c h i n e </w>': 1, 't r a n s l a t i o n , </w>': 1, 'a n d </w>': 3, 'q u e s t i o n </w>': 1, 'a n s w e r i n g . </w>': 1, 'T o p i c s </w>': 1, 'b e </w>': 1, 'c o v e r e d </w>': 1, 'i n c l u d e </w>': 1, 'p a r t - o f - s p e e c h </w>': 1, 't a g g i n g , </w>': 1, 'n - g r a m </w>': 1, 'l a n g u a g e </w>': 2, 'm o d e l l

In [7]:
"""
这个函数遍历词汇表中的所有单词，并计算彼此相邻的一对标记。

EXAMPLE:
    word = 'T h e <\w>'
    这个单词可以两两组合成： [('T', 'h'), ('h', 'e'), ('e', '<\w>')]
    
输入:
    vocab: Dict[str, int]  # vocab统计了词语出现的词频
    
输出:
    pairs: Dict[Tuple[str, str], int] # 字母对，pairs统计了单词对出现的频率
"""
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    
    for word,freq in vocab.items():
        
        # 遍历每一个word里面的symbol，去凑所有的相邻两个内容
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i],symbols[i+1])] += freq

    return pairs
my_pairs = get_stats(my_vocab)
print(my_pairs)

# 好像还是挺简单的！

defaultdict(<class 'int'>, {('T', 'h'): 2, ('h', 'e'): 4, ('e', '</w>'): 16, ('a', 'i'): 2, ('i', 'm'): 1, ('m', 's'): 2, ('s', '</w>'): 10, ('f', 'o'): 5, ('o', 'r'): 8, ('r', '</w>'): 4, ('t', 'h'): 5, ('h', 'i'): 2, ('i', 's'): 3, ('s', 'u'): 1, ('u', 'b'): 1, ('b', 'j'): 1, ('j', 'e'): 1, ('e', 'c'): 2, ('c', 't'): 2, ('t', '</w>'): 3, ('s', 't'): 4, ('t', 'u'): 2, ('u', 'd'): 3, ('d', 'e'): 6, ('e', 'n'): 2, ('n', 't'): 3, ('t', 's'): 3, ('t', 'o'): 2, ('o', '</w>'): 2, ('e', 'v'): 1, ('v', 'e'): 3, ('e', 'l'): 2, ('l', 'o'): 1, ('o', 'p'): 3, ('p', '</w>'): 2, ('a', 'n'): 11, ('n', '</w>'): 9, ('u', 'n'): 1, ('n', 'd'): 5, ('e', 'r'): 4, ('r', 's'): 3, ('t', 'a'): 4, ('d', 'i'): 3, ('i', 'n'): 18, ('n', 'g'): 13, ('g', '</w>'): 4, ('o', 'f'): 3, ('f', '</w>'): 2, ('m', 'a'): 3, ('a', 'l'): 3, ('l', 'g'): 1, ('g', 'o'): 1, ('r', 'i'): 2, ('i', 't'): 2, ('h', 'm'): 1, ('u', 's'): 4, ('s', 'e'): 6, ('e', 'd'): 3, ('d', '</w>'): 6, ('n', 'a'): 1, ('a', 't'): 7, ('u', 'r'): 1, ('r', '

####  开始合并高频字符对



In [16]:
"""
EXAMPLE:
    word = 'T h e <\w>'
    pair = ('e', '<\w>')
    word_after_merge = 'T h e<\w>'
    
输入:
    pair: Tuple[str, str] # 需要合并的字符对
    v_in: Dict[str, int]  # 合并前的vocab
    
输出:
    v_out: Dict[str, int] # 合并后的vocab
    
注意:
    当合并word 'Th e<\w>'中的字符对 ('h', 'e')时，'Th'和'e<\w>'字符对不能被合并。
"""
def merge_vocab(pair, v_in):
    v_out = {}
    # 把pair拆开，然后用空格合并起来，然后用\把空格转义
    bigram = re.escape(' '.join(pair))
    # 自定义一个正则规则, (?<!\S)h\ e(?!\S) 只有前面、后面不是非空白字符(\S)(意思前后得是没东西的)，才匹配h\ e，这样就可以把Th\ e<\w>排除在外
    
    # 理解了，这里的意思是匹配的时候一开始没问题，但是随着合并的越来越多，可能会存在Th\ e<\w>这种已经合并过的
    # 所以需要前后都没别的东西的才是需要匹配！！！
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for v in v_in:
        # 遍历当前的vocabulary，找到匹配正则的v时，才用合并的pair去替换变成新的pair new，如果没有匹配上，那就保持原来的。
        # 比如pair当前是'h'和'e'，然后遍历vocabulary，找到符合前后都没有东西只有'h\ e'的时候就把他们并在一起变成'he'
        new = p.sub(''.join(pair),v)  # p.sub是替换匹配对的那一部分，那就对了！！！
        # 然后新的合并的数量就是当前vocabulary里面pair对应的数量
        v_out[new] = v_in[v]  # 表示这个有多少个
    return v_out

def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens


vocab = get_vocab(text)
print("Vocab =", vocab)
print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('==========')

#about 100 merges we start to see common words
num_merges = 100
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    
    # vocabulary里面pair出现次数最高的作为最先合并的pair
    best = max(pairs, key=pairs.get)
    
    # 先给他合并了再说，当然这里不操作也没什么，到merge_vocab里面都一样
    new_token = ''.join(best)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    # add new token to the vocab
    tokens[new_token] = pairs[best]
    # deduct frequency for tokens have been merged
    tokens[best[0]] -= pairs[best]
    tokens[best[1]] -= pairs[best]
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')
    print('vocab, ', vocab)


Vocab = defaultdict(<class 'int'>, {'T h e </w>': 2, 'a i m s </w>': 1, 'f o r </w>': 4, 't h i s </w>': 1, 's u b j e c t </w>': 1, 'i s </w>': 2, 's t u d e n t s </w>': 1, 't o </w>': 2, 'd e v e l o p </w>': 1, 'a n </w>': 1, 'u n d e r s t a n d i n g </w>': 1, 'o f </w>': 2, 't h e </w>': 2, 'm a i n </w>': 1, 'a l g o r i t h m s </w>': 1, 'u s e d </w>': 2, 'i n </w>': 3, 'n a t u r a l l a n g u a g e </w>': 1, 'p r o c e s s i n g , </w>': 1, 'u s e </w>': 2, 'a </w>': 1, 'd i v e r s e </w>': 1, 'r a n g e </w>': 1, 'a p p l i c a t i o n s </w>': 1, 'i n c l u d i n g </w>': 1, 't e x t </w>': 1, 'c l a s s i f i c a t i o n , </w>': 1, 'm a c h i n e </w>': 1, 't r a n s l a t i o n , </w>': 1, 'a n d </w>': 3, 'q u e s t i o n </w>': 1, 'a n s w e r i n g . </w>': 1, 'T o p i c s </w>': 1, 'b e </w>': 1, 'c o v e r e d </w>': 1, 'i n c l u d e </w>': 1, 'p a r t - o f - s p e e c h </w>': 1, 't a g g i n g , </w>': 1, 'n - g r a m </w>': 1, 'l a n g u a g e </w>': 2, 'm o

### 运行时的逻辑？

In [27]:
def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        # 看vocabulary里面的token频率，相当于上面的code中的tokens去除freq为0的
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        # vocab和其对应的tokens
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

def measure_token_length(token):
    
    # 如果token最后四个元素是 < / w >
    if token[-4:] == '</w>':
        # 那就返回除了最后四个之外的长度再加上1(结尾)
        return len(token[:-4]) + 1
    else:
        # 如果这个token里面没有结尾就直接返回当前长度
        return len(token)
# 这个感觉比较重要！！！！ xj
# 如果vocabulary里面找不到要拆分的词，就根据已经有的token现拆
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    # base case，没词进来了，那拆的结果就是空的
    if string == '':
        return []
    # 已有的sorted tokens没有了，那就真的没这个词了？？ 这里不是很明白！！xj 这里有个递归
    if sorted_tokens == []:
        return [unknown_token] * len(string)

    # 记录拆分结果
    string_tokens = []
    
    # iterate over all tokens to find match
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        
        # 自定义一个正则，然后要把token里面包含句号的变成[.]
        token_reg = re.escape(token.replace('.', '[.]'))
        
        # 在当前string里面遍历，找到每一个match token的开始和结束位置，比如string=good，然后token是o，输出[(2,2),(3,3)]?
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        # if no match found in the string, go to next token
        if len(matched_positions) == 0:
            continue
        # 因为要拆分这个词，匹配上的token把这个word拆开了，那就要拿到除了match部分之外的substring，所以这里要拿match的start
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]
        substring_start_position = 0
        
        
        # 如果有匹配成功的话，就会进入这个循环
        for substring_end_position in substring_end_positions:
            # slice for sub-word
            substring = string[substring_start_position:substring_end_position]
            # tokenize this sub-word with tokens remaining 接着用substring匹配剩余的sorted token，因为刚就匹配了一个
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            # 先把sorted token里面匹配上的记下来 -> 这个是出口，每一段记录下来即可 xj
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        # tokenize the remaining string 去除前头的substring，去除已经匹配上的，后面还剩下substring_start_pos到结束的一段substring没看
        remaining_substring = string[substring_start_position:]
        # 接着匹配
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    else:
        # return list of unknown token if no match is found for the string
        string_tokens = [unknown_token] * len(string)
        
    return string_tokens

"""
该函数生成一个所有标记的列表，按其长度（第一键）和频率（第二键）排序。

EXAMPLE:
    token frequency dictionary before sorting: {'natural': 3, 'language':2, 'processing': 4, 'lecture': 4}
    sorted tokens: ['processing', 'language', 'lecture', 'natural']
    
INPUT:
    token_frequencies: Dict[str, int] # Counter for token frequency
    
OUTPUT:
    sorted_token: List[str] # Tokens sorted by length and frequency

"""
def sort_tokens(tokens_frequencies):
    # 对 token_frequencies里面的东西，先进行长度排序，再进行频次，sorted是从低到高所以要reverse
    sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item:(measure_token_length(item[0]),item[1]), reverse=True)
    
    # 然后只要tokens不要频次
    sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

    return sorted_tokens

#display the vocab
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)

#sort tokens by length and frequency
sorted_tokens = sort_tokens(tokens_frequencies)
print("Tokens =", sorted_tokens, "\n")

#print("vocab tokenization: ", vocab_tokenization)

sentence_1 = 'I like natural language processing!'
sentence_2 = 'I like natural languaaage processing!'
sentence_list = [sentence_1, sentence_2]

for sentence in sentence_list:
    
    print('==========')
    print("Sentence =", sentence)
    
    for word in sentence.split():
        word = word + "</w>"

        print('Tokenizing word: {}...'.format(word))
        if word in vocab_tokenization:
            print(vocab_tokenization[word])
        else:
            print(tokenize_word(string=word, sorted_tokens=sorted_tokens, unknown_token='</u>'))



Tokens = ['understanding</w>', 'algorithms</w>', 'language</w>', 'students</w>', 'subject</w>', 'develop</w>', 'ication', 'lation', 'ing,</w>', 'used</w>', 'inclu', 'ing.</w>', 'aims</w>', 'this</w>', 'main</w>', 'ding</w>', 'ation', 'for</w>', 'and</w>', 'The</w>', 'the</w>', 'use</w>', 'assi', 'gram', 'ing</w>', 'natu', 'nts</w>', 'in</w>', 'is</w>', 'to</w>', 'of</w>', 'pro', 'ver', 'ans', 'par', 'an</w>', 'ang', 'ion', 'ed</w>', 'for', 'e</w>', ',</w>', 's</w>', 'in', 'al', 'st', 'op', 'ic', 'de', 'on', 'or', 'me', 'ss', 't</w>', 'cl', 'ma', 'of', 'ec', 'ag', 'nt', 'ar', 'th', 'at', '.</w>', '</w>', 'e', 's', 't', 'r', 'c', 'p', 'l', 'h', 'm', 'a', 'o', '-', 'n', 'd', 'i', 'w', 'g', 'y', 'x', 'f', 'q', 'u', 'T', 'b', 'P', 'k'] 

Sentence = I like natural language processing!
Tokenizing word: I</w>...
['</u>', '</w>']
Tokenizing word: like</w>...
['l', 'i', 'k', 'e</w>']
Tokenizing word: natural</w>...
['natu', 'r', 'al', '</w>']
Tokenizing word: language</w>...
['language</w>']
Tok